[Task 2]    Rhina Kim - 40130779

Imports

In [1]:
import gensim.downloader as api
import pandas as pd
import numpy as np
from IPython.core.display import HTML

Show all available pre-trained dataset from Gensim

In [2]:
available_models = api.info()['models'].keys()
for model in available_models:
    print(model)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


Load pre-trained embedding model

In [3]:
def load_model(model_name):    
    # obtain pre-trained Word2Vec model
    print("...Model Loading...")
    model = api.load(model_name)
    print("...Model Loaded...")
    
    return model

Load dataset

In [4]:
def load_csv(csv_file):
    # Load synonyms.csv dataset
    df = pd.read_csv(csv_file)

    # display data frame
    HTML(df.to_html())
    
    return df

Compute the closest synonym for each word in loaded csv dataset using Similarity method

In [5]:
def details(df, model):
    # new dataframe created to store closest synonyms with the question word 
    # which is to be found from system model
    details_df = pd.DataFrame(columns=['question', 'answer', 'guess', 'label'])

    # iterate each data in dataframe by rows
    for index, row in df.iterrows():
        
        # create dictionary to store accuracy value 
        # for similarity between question-word and each available guess-words
        accuracy_dict = {}
        
        # assign each elements in row to appropriate variables
        question = row["question"]
        answer = row["answer"]
        guess_0 = row["0"]
        guess_1 = row["1"]
        guess_2 = row["2"]
        guess_3 = row["3"]
        
        # if question-word and answer-word from synonyms.csv found in word2vec model:
        if (question in model) and (answer in model):
            if guess_0 in model:
                accuracy = model.similarity(question, guess_0)  # calculate cosine similarity
                accuracy_dict[guess_0] = accuracy               # append both word and accuracy to dictionary
            if guess_1 in model:
                accuracy = model.similarity(question, guess_1)
                accuracy_dict[guess_1] = accuracy
            if guess_2 in model:
                accuracy = model.similarity(question, guess_2)
                accuracy_dict[guess_2] = accuracy
            if guess_3 in model:
                accuracy = model.similarity(question, guess_3)
                accuracy_dict[guess_3] = accuracy
            
            # sort the accuracy dictionary by keys with higher accuracy value (descending)
            sorted_accuracy_tuples = sorted(accuracy_dict.items(), key=lambda item: item[1], reverse=True)
            sorted_accuracy_dict = {key: value for key, value in sorted_accuracy_tuples}
            #print(sorted_accuracy_dict)
            
            # obtain the guess word with highest accuracy value (first element in sorted dictionary!)
            guess_word = next(iter(sorted_accuracy_dict))
            
            # check if word guessed by system is equal to the answer in synonyms.csv:
            if guess_word == answer:
                label = "correct"
            else:
                label = "wrong"
        
        # else if question-word and answer-word not found in word2vec model:
        else:
            label = "guess"
            guess_word = ""
        
        # append all the data (question-word, answer-word, guess-word by system, label) to new dataframe
        details_df.loc[index] = [question, answer, guess_word, label]

    return details_df

Store model info and csv computed dataset insights

In [6]:
def analysis(details_df, model, model_name):
    # new dataframe created to system model analysis
    analysis_df = pd.DataFrame(columns=['model_name', 'corpus_size', '#correct_labels', '#questions_answered_no_guess', 'accuracy_model'])

    vocab_size = len(model)     # size of the vocabulary(corpus) in model
    num_labels_dict = details_df["label"].value_counts()    # returns dictionary with frequency elements in all labels
    num_correct_labels = num_labels_dict["correct"]         # number of all elements with correct label
    V = num_labels_dict["correct"] + num_labels_dict["wrong"]   # number of all elements with no guess label
    accuracy_model_val = num_correct_labels / V             # accuracy model value (C/V)

    # append all the data (model_name, corpus_size, #correct_labels, #questions_answered_no_guess, accuracy_model) 
    # to new dataframe
    analysis_df.loc[0] = [model_name, vocab_size, num_correct_labels, V, accuracy_model_val]
    
    return analysis_df

Save data obtained to csv files

In [7]:
def save_csv(details_df, analysis_df, model_name):
    directory = "../outputs/"
    
    # save model labels to -details.csv
    file_name_details = directory + model_name + "-details.csv"
    details_df.to_csv(file_name_details, index=False)

    # save model analysis to -analysis.csv
    file_name_analysis = directory + "analysis.csv"
    analysis_df.to_csv(file_name_analysis, mode='a', index=False)

In [8]:
# Task 1: Load "word2vec-google-news-300" word2vec pre-trained model
# Task 2: Load 4 other English any pre-trained models
#   -   2 new models from different corpora but same embedding size
#   -   2 new models from the same corpus but different embedding sizes
model_names = [ 
    ["glove-twitter-200", "glove-wiki-gigaword-200"],
    ["glove-twitter-25", "glove-twitter-50"]
]

# Load synonyms.csv dataset
df = load_csv("synonyms.csv")

# Load pre-trained model, compute similarity, and output results to csv files
for index, task in enumerate(model_names):
    if index == 0:
        print("======== Executing Task 2(1): different corpora & same embedding size ========")
    else:
        print("======== Executing Task 2(2): same corpus & different embedding size ========")
    for model_name in task:
        print("--> '" + model_name + "'")
        
        # Load pre-trained model
        model = load_model(model_name)

        # create details dataframe which computes the 
        # closest synonym for each word in loaded csv dataset
        details_df = details(df, model)
        # display new dataframe
        HTML(details_df.to_html())

        # Store model info and csv computed dataset insights
        analysis_df = analysis(details_df, model, model_name)
        # display new dataframe
        HTML(analysis_df.to_html())

        # save both data to csv files
        save_csv(details_df, analysis_df, model_name)

# Compare the results

======== Executing Task 2(1): different corpora & same embedding size ========
--> 'glove-twitter-200'
...Model Loading...
